In [ ]:
from collections import defaultdict
from dataclasses import dataclass
from immutables import Map
import re
from typing import Any, Callable, Iterable, Mapping

In [ ]:
# SCRIPT: str = clipboard.raw.replace("\r\n", "\n")
script: str = SCRIPT

In [ ]:
REGISTERS = "wxyz"
RE_PART_REGISTER = "[" + REGISTERS + "]"
RE_PART_NUMBER = r"-?\d+"
RE_PART_REGORNUM = RE_PART_REGISTER + "|" + RE_PART_NUMBER
def abfunc(name: str) -> re.Pattern[str]:
    return re.compile(name + r" (?P<a>" + RE_PART_REGISTER + r") (?P<b>" + RE_PART_REGORNUM + r")")

RE_INP = re.compile(r"inp (?P<a>" + RE_PART_REGISTER + r")")
RE_ADD = abfunc("add")
RE_MUL = abfunc("mul")
RE_DIV = abfunc("div")
RE_MOD = abfunc("mod")
RE_EQL = abfunc("eql")

```
div var 1
```
won't do anything.

In [ ]:
RE_DIV1 = re.compile(
    r"div [wxyz] 1\n",
    flags=re.MULTILINE,
)

def replace_div1(text: str):
    return RE_DIV1.sub(r"", text)

script = replace_div1(script)

```
mul var 0
add var value
```
means store `value` to `var`

In [ ]:
RE_MUL0ADD = re.compile(
    r"^mul (?P<var>[wxyz]) 0\n"
    r"add (?P=var) (?P<value>\w+)\n",
    flags=re.MULTILINE,
)

def replace_mul0add(text: str):
    return RE_MUL0ADD.sub(r"SET \g<var> \g<value>\n", text)

RE_C_SET = abfunc("SET")

script = replace_mul0add(script)

```
eql var value
eql var 0
```
means test equality, then test that the last equality failed; it is a non-equality (`!=`) test

In [ ]:
RE_EQLEQL0 = re.compile(
    r"^eql (?P<var>[wxyz]) (?P<value>\w+)\n"
    r"eql (?P=var) 0\n",
    flags=re.MULTILINE,
)

def replace_eqleql0(text: str):
    return RE_EQLEQL0.sub(r"NEQ \g<var> \g<value>\n", text)

RE_C_NEQ = abfunc("NEQ")

script = replace_eqleql0(script)

Check for no operations on `w` other than `inp`

In [ ]:
RE_OP_ON_W = re.compile(r"^(?!inp)\w+ w", flags=re.MULTILINE)
assert RE_OP_ON_W.search(script) is None

Therefore, any `eql` out of range (`w` is `1..9`) will fail (and `NEQ` succeed).

In [ ]:
RE_ADDHIGH_EQL_W = re.compile(
    r"^(?P<opmod>mod (?P<var>[wxyz]) \d+)\n"
    r"(?P<opadd>add (?P=var) \d{2,})\n"
    r"(?P<opeql>eql|NEQ) (?P=var) w\n",
    flags=re.MULTILINE,
)

def replace_addhigh_eql_w_occurance(m: re.Match[str]) -> str:
    value = "1" if m["opeql"] == "NEQ" else "0"
    return f"SET {m['var']} {value}\n"

def replace_addhigh_eql_w(text: str):
    return RE_ADDHIGH_EQL_W.sub(replace_addhigh_eql_w_occurance, text)

script = replace_addhigh_eql_w(script)

All patterns either have a `DIV z 26` and a `NEQ x w`, or no `DIV` (originally `DIV z 1`) and a `SET x 1` (originally out-of-range `NEQ`).
All patterns have two variable numbers: `add x [k]` and `add y [n]`.

```
ALL SECTIONS MATCH
    inp w\n
    SET x z\n
    (mod x 26\n
    div z 26\n
    add x (-?\d+)\n # "k"
    NEQ x w|SET x 1)\n # (mod div add NEQ)|set (set for k>9)
    SET y 25\n
    mul y x\n
    add y 1\n
    mul z y\n
    SET y w\n
    add y (\d+)\n # "n"
    mul y x\n
    add z y


mod, div, add, neq:
                x = z => x = zlast
                x %= 26 => x = zlast % 26 => {zlast >= 0}
    z //= 26 => z = zlast // 26
                x += k => x = (zlast%26) + k
                x = x!=w => x = 1 if ((zlast%26)+k!=w) else 0
            y = 25
            y *= x => y = 25 if ((zlast%26)+k!=w) else 0
            y += 1 => y = 26 if ((zlast%26)+k!=w) else 1
    z *= y => z = (zlast//26)*(26 if ((zlast%26)+k!=w) else 1)
        y = w
        y += n => y = w + n
        y *= x => y = (w+n) if ((zlast%26)+k!=w) else 0
    z += y => z = (zlast//26)*(26 if ((zlast%26)+k!=w) else 1) + ((w+n) if ((zlast%26)+k!=w) else 0)
                = ((zlast//26)*26 if ((zlast%26)+k!=w) else (zlast//26)) + ((w+n) if ((zlast%26)+k!=w) else 0)
                = ((zlast//26)*26 + w+n if ((zlast%26)+k!=w) else (zlast//26))
set:
                x = 1
            y = 25
            y *= x => y = 25
            y += 1 => y = 26
    z *= y => z = zlast * 26
        y = w
        y += n => y = w + n
        y *= x => y = w + n
    z += y => z = zlast*26 + w+n
```

In [ ]:
RE_SECTION = re.compile(r"inp w\nSET x z\n(?P<diff>mod x 26\ndiv z 26\nadd x (?P<k>-?\d+)\nNEQ x w|SET x 1)\nSET y 25\nmul y x\nadd y 1\nmul z y\nSET y w\nadd y (?P<n>\d+)\nmul y x\nadd z y")


class BaseSection:
    n: int

    def calculate(self, w: int, lastz: int) -> int:
        raise NotImplementedError

    @staticmethod
    def from_match(m: re.Match[str]) -> "BaseSection":
        if "SET" in m["diff"]:
            return SetSection(int(m["n"]))
        else:
            return NeqSection(k=int(m["k"]), n=int(m["n"]))

    def inverse(self, z: int) -> Iterable[tuple[int, int]]:
        raise NotImplementedError


@dataclass
class NeqSection(BaseSection):
    n: int
    k: int

    def calculate(self, w: int, lastz: int) -> int:
        assert lastz >= 0
        return (int(lastz/26)*26 + w + self.n) if ((lastz%26) + self.k) != w else int(lastz/26)

    def inverse(self, z: int) -> Iterable[tuple[int, int]]:
        for w in range(1, 10):
            # (lastz%26) + self.k != w
            # => (int(lastz/26)*26 + w + self.n)
            nonmod_lastz = z - w - self.n
            if nonmod_lastz >= 0 and nonmod_lastz % 26 == 0:
                for mod in range(26):
                    if mod + self.k != w:
                        yield nonmod_lastz + mod, w

            # (lastz%26) + self.k == w
            # => int(lastz/26)
            nonmod_lastz = z * 26
            mod = w - self.k  # (lastz%26) + self.k == w
            if nonmod_lastz >= 0 and 0 <= mod < 26:
                yield nonmod_lastz + mod, w


@dataclass
class SetSection(BaseSection):
    n: int

    def calculate(self, w: int, lastz: int) -> int:
        return lastz*26 + w + self.n

    def inverse(self, z: int) -> Iterable[tuple[int, int]]:
        w = (z % 26) - self.n
        if 1 <= w <= 9:
            yield z // 26, w


sections = [BaseSection.from_match(m) for m in RE_SECTION.finditer(script)]
assert len(sections) == 14


In [ ]:
def run(func: Callable[[Iterable[tuple[int, ...]]], tuple[int, ...]]) -> str:
    nums: dict[int, tuple[int, ...]] = {0: ()}

    for sectionnum, section in reversed(list(enumerate(sections))):
        new_nums: defaultdict[int, set[tuple[int, ...]]] = defaultdict(set)
        for n, stack in nums.items():
            for new_n, w in section.inverse(n):
                if section.calculate(w, new_n) != n:
                    print(w, new_n, n)
                if sectionnum == 0 and new_n != 0:
                    continue
                new_nums[new_n].add((w,) + stack)
        nums = {i: func(s) for i, s in new_nums.items()}
        # print(sectionnum, section, len(nums))
    
    assert len(nums) == 1
    return "".join(map(str, next(iter(nums.values()))))

print("Part 1:", run(max))
print("Part 2:", run(min))

In [ ]:
# def _combinable(k1: Map[str, int], k2: Map[str, int]) -> bool:
#     if k1 == k2:
#         return True
#     for key in set(k1) & set(k2):
#         if k1[key] != k2[key]:
#             return False
#     return True

# def _combine(k1: Map[str, int], k2: Map[str, int]) -> Map[str, int]:
#     if k1 == k2:
#         return k1
#     assert _combinable(k1, k2)
#     return Map(dict(k1) | dict(k2))


# def _sort_dict(d: Mapping[str, int]) -> Iterable[tuple[str, int]]:
#     return sorted(d.items())


# @dataclass(frozen=True)
# class SetOfValues:
#     values: Map[Map[str, int], int]

#     def __repr__(self) -> str:
#         if self.dimensions() == 1:
#             return "SetOfValues{" + next(iter(next(iter(self.values)))) + " = " + ", ".join(
#                 ";".join(f"{kv}" for kv in k.values())
#                 + f"=>{v}" for k, v in sorted(self.values.items(), key=lambda pair: tuple(pair[0].values()))
#             ) + "}"
#         return "SetOfValues{" + ", ".join(
#             ";".join(f"{kk}={kv}" for kk, kv in sorted(k.items()))
#             + f" => {v}" for k, v in sorted(self.values.items(), key=lambda pair: tuple(_sort_dict(pair[0])))
#         ) + "}"

#     def dimensions(self) -> int:
#         if not len(self.values):
#             raise ValueError
#         empty: set[str] = set()
#         return len(empty.union(*(set(m) for m in self.values)))

#     @classmethod
#     def from_mapping(cls, values: Mapping[Map[str, int], int]):
#         return cls(Map(values))

#     @classmethod
#     def from_iterable(cls, var: str, values: Iterable[int]):
#         return cls(Map({Map({var: v}): v for v in values}))

#     def __add__(self, obj: Any) -> "SetOfValues":
#         if isinstance(obj, int):
#             return SetOfValues.from_mapping({
#                 k: self.values[k] + obj for k in self.values
#             })
#         elif isinstance(obj, SetOfValues):
#             return SetOfValues.from_mapping({
#                 _combine(k1, k2): self.values[k1] + obj.values[k2]
#                 for k1 in self.values for k2 in obj.values
#                 if _combinable(k1, k2)
#             })
#         else:
#             return NotImplemented

#     def __radd__(self, obj: Any) -> "SetOfValues":
#         return self + obj

#     def __mul__(self, obj: Any) -> "SetOfValues":
#         if isinstance(obj, int):
#             return SetOfValues.from_mapping({
#                 k: self.values[k] * obj for k in self.values
#             })
#         elif isinstance(obj, SetOfValues):
#             return SetOfValues.from_mapping({
#                 _combine(k1, k2): self.values[k1] * obj.values[k2]
#                 for k1 in self.values for k2 in obj.values
#                 if _combinable(k1, k2)
#             })
#         else:
#             return NotImplemented

#     def __rmul__(self, obj: Any) -> "SetOfValues":
#         return self * obj

#     def __mod__(self, obj: Any) -> "SetOfValues":
#         if isinstance(obj, int):
#             return SetOfValues.from_mapping({
#                 k: self.values[k] % obj for k in self.values
#             })
#         elif isinstance(obj, SetOfValues):
#             return SetOfValues.from_mapping({
#                 _combine(k1, k2): self.values[k1] % obj.values[k2]
#                 for k1 in self.values for k2 in obj.values
#                 if _combinable(k1, k2)
#             })
#         else:
#             return NotImplemented

#     def __rmod__(self, obj: Any) -> "SetOfValues":
#         if isinstance(obj, int):
#             return SetOfValues.from_mapping({
#                 k: obj % self.values[k] for k in self.values
#             })
#         return NotImplemented

#     def __floordiv__(self, obj: Any) -> "SetOfValues":
#         if isinstance(obj, int):
#             return SetOfValues.from_mapping({
#                 k: self.values[k] // obj for k in self.values
#             })
#         elif isinstance(obj, SetOfValues):
#             return SetOfValues.from_mapping({
#                 _combine(k1, k2): self.values[k1] // obj.values[k2]
#                 for k1 in self.values for k2 in obj.values
#                 if _combinable(k1, k2)
#             })
#         else:
#             return NotImplemented

#     def __rfloordiv__(self, obj: Any) -> "SetOfValues":
#         if isinstance(obj, int):
#             return SetOfValues.from_mapping({
#                 k: obj // self.values[k] for k in self.values
#             })
#         return NotImplemented

#     def check_eq(self, obj: "int | SetOfValues") -> "SetOfValues":
#         if isinstance(obj, int):
#             return SetOfValues.from_mapping({
#                 k: (1 if self.values[k] == obj else 0)
#                 for k in self.values
#             })
#         return SetOfValues.from_mapping({
#             _combine(k1, k2): (1 if self.values[k1] == obj.values[k2] else 0)
#             for k1 in self.values for k2 in obj.values
#             if _combinable(k1, k2)
#         })

#     def invert_as_bool(self) -> "SetOfValues":
#         return SetOfValues.from_mapping({
#             k: (0 if self.values[k] else 1) for k in self.values
#         })

#     def compact_max_values(self) -> "SetOfValues":
#         max_values: dict[int, Map[str, int]] = dict()
#         for k, v in self.values.items():
#             if v not in max_values or tuple(_sort_dict(k)) > tuple(_sort_dict(max_values[v])):
#                 max_values[v] = k
#         return SetOfValues.from_mapping({k: v for v, k in max_values.items()})

#     def compact(self) -> "SetOfValues | int":
#         if len(set(self.values.values())) == 1:
#             return next(iter(self.values.values()))

#         effective: set[str] = set()
#         key_values: defaultdict[str, dict[Map[str, int], int]] = defaultdict(dict)
#         for m, i in self.values.items():
#             for k in m:
#                 if k in effective:
#                     continue
#                 mapping = m.delete(k)
#                 if mapping in key_values[k]:
#                     if key_values[k][mapping] != i:
#                         key_values.pop(k)
#                         effective.add(k)
#                 else:
#                     key_values[k][mapping] = i
#         return SetOfValues.from_mapping({
#             Map({kk: kv for kk, kv in k.items() if kk in effective}): v
#             for k, v in self.values.items()
#         }).compact_max_values()


# def compare_eql(v1: int | SetOfValues, v2: int | SetOfValues) -> int | SetOfValues:
#     if v1 == v2:
#         return 1
#     if isinstance(v1, SetOfValues):
#         return v1.check_eq(v2)
#     if isinstance(v2, SetOfValues):
#         return v2.check_eq(v1)
#     return 0


# def compare_neq(v1: int | SetOfValues, v2: int | SetOfValues) -> int | SetOfValues:
#     eql = compare_eql(v1, v2)
#     if isinstance(eql, SetOfValues):
#         return eql.invert_as_bool()
#     return (0 if eql else 1)


# def restrict(case: Callable[[int], bool], value: int | SetOfValues) -> int | SetOfValues:
#     if isinstance(value, int):
#         assert case(value)
#         return value
#     restricted = SetOfValues.from_mapping({k: v for k, v in value.values.items() if case(v)})
#     assert restricted == value
#     return restricted


# def div(a: int | SetOfValues, b: int | SetOfValues):
#     if isinstance(a, SetOfValues):
#         if isinstance(b, SetOfValues):
#             return SetOfValues.from_mapping({
#                 _combine(k1, k2): int(a.values[k1] / b.values[k2])
#                 for k1 in a.values for k2 in b.values
#                 if _combinable(k1, k2)
#             })
#         else:
#             return SetOfValues.from_mapping({
#                 k: int(a.values[k] / b) for k in a.values
#             })
#     else:
#         if isinstance(b, SetOfValues):
#             return SetOfValues.from_mapping({
#                 k: int(a / b.values[k]) for k in b.values
#             })
#         else:
#             return int(a / b)


In [ ]:
# text: list[str] = []
# i = 0
# guessing = True
# variables: dict[str, SetOfValues | int] = dict(w=SetOfValues.from_iterable("UNSET", (0,)), x=0, y=0, z=0)

# def var_or_int(v: str) -> SetOfValues | int:
#     if v in variables:
#         return variables[v]
#     return int(v)

# def var_str() -> str:
#     return ", ".join(f"{k} = {variables[k]}" for k in sorted(variables))

# def int_eq(v: Any, test: int) -> bool:
#     return isinstance(v, int) and v == test

# def dimensions(v: SetOfValues | int) -> int:
#     if isinstance(v, SetOfValues):
#         return v.dimensions()
#     return 0

# text.append("INITIAL:")
# text.append("  " + var_str())
# text.append("")

# for linenum, line in enumerate(script.splitlines(), 1):
#     if (m := RE_INP.fullmatch(line)):
#         di = f"d{i:02}"
#         text.append(di)
#         variables[m["a"]] = SetOfValues.from_iterable(di, range(1, 10))
#         i += 1

#         z = variables["z"]
#         if isinstance(z, SetOfValues):
#             variables["z"] = SetOfValues.from_mapping({k: v for k, v in z.values.items()})
#     elif (m := RE_ADD.fullmatch(line)):
#         a, b = variables[m["a"]], var_or_int(m["b"])
#         if int_eq(b, 0):
#             continue
#         variables[m["a"]] = a + b
#     elif (m := RE_MUL.fullmatch(line)):
#         a, b = variables[m["a"]], var_or_int(m["b"])
#         if int_eq(b, 1):
#             continue
#         variables[m["a"]] = a * b
#     elif (m := RE_DIV.fullmatch(line)):
#         a, b = variables[m["a"]], var_or_int(m["b"])
#         if int_eq(b, 1):
#             continue
#         rb = restrict(lambda v: v != 0, b)
#         variables[m["a"]] = div(a, rb)
#     elif (m := RE_MOD.fullmatch(line)):
#         a, b = variables[m["a"]], var_or_int(m["b"])
#         ra = restrict(lambda v: v >= 0, a)
#         rb = restrict(lambda v: v > 0, b)
#         variables[m["a"]] = ra % rb
#     elif (m := RE_EQL.fullmatch(line)):
#         a, b = variables[m["a"]], var_or_int(m["b"])
#         variables[m["a"]] = compare_eql(a, b)
#     elif (m := RE_C_SET.fullmatch(line)):
#         variables[m["a"]] = var_or_int(m["b"])
#     elif (m := RE_C_NEQ.fullmatch(line)):
#         a, b = variables[m["a"]], var_or_int(m["b"])
#         variables[m["a"]] = compare_neq(a, b)
#     else:
#         raise ValueError(f"Unrecognised line: {line}")

#     for k, v in variables.items():
#         if isinstance(v, SetOfValues):
#             variables[k] = v.compact()

#     z = variables["z"]
#     print(f"{linenum:3}/{len(script.splitlines()):3}", line, f"({dimensions(variables['w'])}/{dimensions(variables['x'])}/{dimensions(variables['y'])}/{dimensions(variables['z'])} dimension(s), z:{len(z.values) if isinstance(z, SetOfValues) else 1})")
#     text.append("  " + line)
#     # if guessing:
#     #     text.append("    " + var_str())


In [ ]:
# {k: v if isinstance(v, int) else (len(v.values), 0 in v.values.values()) for k, v in variables.items()}

In [ ]:
# formatted_script = "\n".join(text)
# clipboard(formatted_script + "\n")
# clipboard(script + "\n")
# print(script)
pass